In [1]:
import pandas as pd
from Config import config
import glob
from tqdm.auto import tqdm
from itertools import combinations

In [2]:
candidate_set_df = pd.read_csv("/workspace/clustering/outputs/results/NZL_final_clustered.csv")
df_raw = pd.read_csv(config.raw_dir + f"Fuse_NZL.csv",engine='c')
golden_df = pd.read_csv(config.gt_dir + f"GT_NZL.csv")

In [4]:
matches = set()
for cluster_id, values in tqdm(candidate_set_df.groupby(["clusterId"])):
    names = values["placeId"].tolist()
    comb = set([tuple(sorted(x)) for x in set(combinations(names, 2))])
    matches.update(comb)
df_pairs = pd.DataFrame(matches)
df_pairs.columns = ['placeId1', 'placeId2']

gold = []
for i, row in golden_df.iterrows():
    gold.append(sorted([row["placeId1"], row["placeId2"]]))
gold = pd.DataFrame(gold)
gold.columns = ['placeId1', 'placeId2']

  0%|          | 0/302594 [00:00<?, ?it/s]

In [8]:
gold = gold.drop_duplicates()

In [9]:
df_pairs["detected"] ="yes"

In [10]:
merged_df = pd.merge(gold,df_pairs,how ='left',on=['placeId1', 'placeId2'],suffixes=["","r"])

In [16]:
not_detected_df = merged_df[merged_df["detected"].isnull()][["placeId1","placeId2"]]
not_detected_df.drop_duplicates(inplace=True)

In [17]:
not_detected_df = pd.merge(not_detected_df, df_raw,  how='left', left_on=[
                    'placeId1'], right_on=['placeId'])
not_detected_df.drop('placeId',inplace=True,axis=1)
not_detected_df = pd.merge(not_detected_df, df_raw,  how='left', left_on=[
                    'placeId2'], right_on=['placeId'],suffixes=["1","2"])
not_detected_df.drop('placeId',inplace=True,axis=1)

In [39]:
cols = ['placeId1', 'placeId2', 'sourceNames1', 'sourceNames2', 'latitude1',
    'latitude2', 'longitude1', 'longitude2', 'category1', 'category2',
    'cities1', 'cities2', 'streets1', 'streets2', 'brands1', 'brands2',
    'email1', 'email2', 'houseNumber1', 'houseNumber2', 'insertedCategories1',
    'insertedCategories2', 'internet1', 'internet2', 'phoneNumbers1',
    'phoneNumbers2', 'postalCode1', 'postalCode2', 'preemptiveCategories1',
    'preemptiveCategories2', 'rawCategories1', 'rawCategories2',
    'subCategory1', 'subCategory2', 'clusterId1', 'clusterId2'
]


In [ ]:
not_detected_df[cols].to_csv(f"Missed_entitymatching_{config.country}.csv",index=None)

In [27]:
gold["original"] = "yes"

In [31]:
merged1 = pd.merge(df_pairs,gold,how ='left',on=['placeId1', 'placeId2'],suffixes=["","r"])

In [35]:
additional_detected_df = merged1[merged1["original"].isnull()][["placeId1","placeId2"]]
additional_detected_df.drop_duplicates(inplace=True)

In [37]:
additional_detected_df = pd.merge(additional_detected_df, df_raw,  how='left', left_on=[
                    'placeId1'], right_on=['placeId'])
additional_detected_df.drop('placeId',inplace=True,axis=1)
additional_detected_df = pd.merge(additional_detected_df, df_raw,  how='left', left_on=[
                    'placeId2'], right_on=['placeId'],suffixes=["1","2"])

In [40]:
additional_detected_df[cols].to_csv(f"addition_entitymatched_{config.country}.csv",index=None)